In [34]:
import pandas as pd
import numpy as np

import folium
from folium.plugins import AntPath

from pprint import pprint
from pprint import PrettyPrinter

In [35]:
df_intercambio = pd.read_csv('../Obtencion datos/intercambio_electrico.csv')

df_intercambio = df_intercambio.drop(columns=['Unnamed: 0'])

df_intercambio

,nombre,tipo de intercambio,Valores,Porcentaje,Fecha actualización
0,Francia,Exportación,-19961.300,0.330618,08/09/2023
1,Francia,Exportación,-15923.134,0.243451,09/09/2023
2,Francia,Exportación,-21133.600,0.315685,10/09/2023
3,Francia,Exportación,-11122.500,0.202408,11/09/2023
4,Francia,Exportación,-18856.900,0.299353,12/09/2023
...,...,...,...,...,...
326,Andorra,saldo,-434.000,1.000000,04/10/2023
327,Andorra,saldo,-410.000,1.000000,05/10/2023
328,Andorra,saldo,-437.000,1.000000,06/10/2023
329,Andorra,saldo,-399.000,1.000000,07/10/2023


In [36]:
df_intercambio.columns

Index(['nombre', 'tipo de intercambio', 'Valores', 'Porcentaje',
       'Fecha actualización'],
      dtype='object')

In [37]:
df_intercambio['nombre'].unique()

array(['Francia', 'Portugal', 'Marruecos', 'Andorra'], dtype=object)

In [38]:
df_intercambio['tipo de intercambio'].unique()

array(['Exportación', 'Importación', 'saldo'], dtype=object)

In [39]:
españa_alt = 40.4637
españa_lat = -3.7492

In [40]:
#Creacion del mapa

from IPython.display import display

spain_map = folium.Map(location = [españa_alt, españa_lat], 
                       zoom_start= 5, 
                       tiles = 'Esri Worldimagery', 
                       width='500px', 
                       height='500px'
                       )

bounds =   [[51.1242, -17.0000],[20.0000, 9.6625]]

spain_map.fit_bounds(bounds)


display(spain_map)


In [41]:
#Coordenadas de los paises donde existe un intercambio energetico.
lista_paises = ['Marruecos', 'Francia', 'Portugal', 'Andorra', 'España']
lista_altitudes = [31.83999, 46.57771, 39.68023, 42.5462, 40.4637]
lista_latitudes = [-6.19721, 2.78159, -8.80606, 1.5034, -3.7492]


df_coordenadas = pd.DataFrame()
df_coordenadas['nombre'] = lista_paises
df_coordenadas['altitud'] = lista_altitudes
df_coordenadas['latitud'] = lista_latitudes

df_coordenadas

,nombre,altitud,latitud
0,Marruecos,31.83999,-6.19721
1,Francia,46.57771,2.78159
2,Portugal,39.68023,-8.80606
3,Andorra,42.54620,1.50340
4,España,40.46370,-3.74920


In [42]:
marruecos = [df_coordenadas['altitud'][0], df_coordenadas['latitud'][0]]
francia = [df_coordenadas['altitud'][1], df_coordenadas['latitud'][1]]
portugal = [df_coordenadas['altitud'][2], df_coordenadas['latitud'][2]]
andorra = [df_coordenadas['altitud'][3], df_coordenadas['latitud'][3]]
españa = [df_coordenadas['altitud'][4], df_coordenadas['latitud'][4]]

In [43]:
df_unido = pd.merge(df_intercambio, df_coordenadas, how = 'left', on = 'nombre')


In [44]:
df_unido

,nombre,tipo de intercambio,Valores,Porcentaje,Fecha actualización,altitud,latitud
0,Francia,Exportación,-19961.300,0.330618,08/09/2023,46.57771,2.78159
1,Francia,Exportación,-15923.134,0.243451,09/09/2023,46.57771,2.78159
2,Francia,Exportación,-21133.600,0.315685,10/09/2023,46.57771,2.78159
3,Francia,Exportación,-11122.500,0.202408,11/09/2023,46.57771,2.78159
4,Francia,Exportación,-18856.900,0.299353,12/09/2023,46.57771,2.78159
...,...,...,...,...,...,...,...
326,Andorra,saldo,-434.000,1.000000,04/10/2023,42.54620,1.50340
327,Andorra,saldo,-410.000,1.000000,05/10/2023,42.54620,1.50340
328,Andorra,saldo,-437.000,1.000000,06/10/2023,42.54620,1.50340
329,Andorra,saldo,-399.000,1.000000,07/10/2023,42.54620,1.50340


In [45]:
#Creación de emojis por país

#Si no está instalado, pip install emoji
import emoji

esp_emoji = emoji.emojize(':Spain:')
mar_emoji = emoji.emojize(":Morocco:")
fra_emoji = emoji.emojize(':France:')
por_emoji = emoji.emojize(':Portugal:')
and_emoji = emoji.emojize(':Andorra:')


In [46]:
esp_emoji

'🇪🇸'

In [47]:
intercambios = folium.map.FeatureGroup(name='Intercambios')

input_tipo_inter = input('Introduce el tipo de intercambio: importación, exportación, o saldo: ______ ').lower()

df_filtrado = df_unido[df_unido['tipo de intercambio'].str.lower() == input_tipo_inter]


for lat, lng, pais, valores, porcentaje, fecha in zip(df_filtrado['altitud'], 
                           df_filtrado['latitud'], 
                           df_filtrado['nombre'],  
                           df_filtrado['Valores'], 
                           df_filtrado['Porcentaje'],
                           df_filtrado['Fecha actualización']):

        contenido_label = f'''<b> Pais: {pais} </b><br> 
                            <b>Tipo de intercambio: {input_tipo_inter} </b><br> 
                            <b>Valores: {valores} </b><br> 
                            <b>Porcentaje: {porcentaje} </b><br>
                            <b>Fecha actualización: {fecha} </b>'''
        intercambios.add_child(folium.Marker(location=[lat, lng], 
                                        popup=contenido_label))



spain_map.add_child(intercambios)


folium.Marker(
        location= españa,
        icon=folium.DivIcon(
                html = f'<div style="font-size: 50px;" >{esp_emoji}</div>'),
        popup = 'España'
        ).add_to(spain_map)

folium.Marker(
        location= portugal,
        icon=folium.DivIcon(
                html = f'<div style="font-size: 30px;" >{por_emoji}</div>'),
        popup = 'Portugal'
        ).add_to(spain_map)
folium.Marker(
        location= francia,
        icon=folium.DivIcon(
                html = f'<div style="font-size: 30px;" >{fra_emoji}</div>'),
        popup = 'Francia'
        ).add_to(spain_map)
folium.Marker(
        location= marruecos,
        icon=folium.DivIcon(
                html = f'<div style="font-size: 30px;" >{mar_emoji}</div>'),
        popup = 'Marruecos'
        ).add_to(spain_map)
folium.Marker(
        location= andorra,
        icon=folium.DivIcon(
                html = f'<div style="font-size: 30px;" >{and_emoji}</div>'),
        popup = 'Andorra'
        ).add_to(spain_map)





AntPath(locations = [marruecos, españa],
        color = 'red',
        delay = 1000,
        weight = 5).add_to(spain_map)

AntPath(locations = [francia, españa],
        color = 'blue',
        delay = 2000,
        weight = 5).add_to(spain_map)

AntPath(locations = [andorra, españa],
        color = 'yellow',
        delay = 500,
        weight = 5).add_to(spain_map)

AntPath(locations = [portugal, españa],
        color = 'green',
        delay = 4000,
        weight = 5).add_to(spain_map)

spain_map

In [48]:
# TO-DO;
# CUANDO SEA EXPORTACIÓN - ANTPATH SEA DE ESPAÑA AL PAÍS EXPORTADO.
# CUANDO SEA IMPORTACIÓN - ANTPATH SEA DEL PAIS DE IMPORTACION A ESPAÑA.
# CUANDO SEA SALDO - SI EL NUMERO ES POSITIVO, QUE SEA DE DE FUERA A DENTRO, SI ES NEGATIVO, QUE SEA DE DENTRO A FUERA.


#CREACIÓN MAPA CUYOS LABELS SEAN CIRCULOS CUYO TAMAÑO DEPENDA DE LA ENERGÍA IMPORTADA O EXPORTADA.